This notebook reverses the process of the encoder.

In [1]:
in_path = 'input.mp4'  # path to the encoded .mp4 video file.
out_path = 'output'  # path to the output file decoded from the video.

W, H = 1920, 1080  # width and height of each frame.
S = 4  # squared size of each block.

assert W % S == 0 and H % S == 0, 'width and height need to be divisible by squared size'
assert W % 4 == 0, 'better to have 4-byte alignment to avoid generating padding'

BPF = W*H//(S*S*8) # number of bytes per frame

In [2]:
from PIL import Image
import numpy as np

import subprocess
import shlex
import itertools
import io

In [3]:
def from_binary(a):
    a = a.reshape(-1, 8).T
    return bytes(np.sum([a[i] << 7-i for i in range(8)], 0, 'uint8'))

def extract_im(im):
    # inputs PIL.Image, outputs bytes
    a = np.asarray(im.resize((W // S, H // S), resample=Image.BOX).convert('L'))
    return from_binary(a >= 128)

In [5]:
args = shlex.split(f'ffmpeg -i "{in_path}" -vf fps=30 -c:v bmp -f image2pipe -')
ffmpeg = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL).stdout
out_fp = open(out_path, 'wb')

try:
    for count in itertools.count(1):
        head = ffmpeg.read(6)
        if not head:
            break
            
        size = int.from_bytes(head[2:6], 'little')
        bmp = head + ffmpeg.read(size - 6)
        
        data = extract_im(Image.open(io.BytesIO(bmp)))
        out_fp.write(data)
        
        print(f'Frames Decoded: {count}', end='\r')
    
finally:
    ffmpeg.close()
    out_fp.close()